# Inference Data Mount

In [141]:
!mount -t nfs4 -o nfsvers=4.1,rsize=1048576,wsize=1048576,hard,timeo=600,retrans=2,noresvport 172.31.91.151:/ ./efs_inference_data

mount: exec /Library/Filesystems/nfs4.fs/Contents/Resources/mount_nfs4 for /Users/purgatorid/Documents/GitHub/Project Canopy/cb_feature_detection/inference/efs_inference_data: No such file or directory


# For Docker Run / Sagemaker

In [1]:
import sys
sys.executable

'/usr/local/bin/python'

# Start Local / Sagemaker Imports

In [1]:
import os
import rasterio as rio
import numpy as np
from rasterio.windows import Window
from glob import glob
from shapely.geometry import Polygon
from shapely.geometry import box
import geopandas as gpd
from rasterio.windows import get_data_window
import rasterio as rio
from inference_predict import *
import boto3
import matplotlib.pyplot as plt
# import gdal
from rasterio.enums import Resampling
from rasterio.vrt import WarpedVRT

# Windowing

In [4]:
def get_windows(img_dim, patch_size=(240, 240), stride=(240, 240)):
    patch_size = np.array(patch_size)
    stride = np.array(stride)
    img_dim = np.array(img_dim)
    # to take into account edges, add additional blocks around right side edge and bottom edge of raster
    new_img_dim = [img_dim[0] + stride[0],img_dim[1] + stride[0]]
    
    max_dim = (new_img_dim//patch_size)*patch_size - patch_size

    ys = np.arange(0, img_dim[0], stride[0])
    xs = np.arange(0, img_dim[1], stride[1])

    tlc = np.array(np.meshgrid(ys, xs)).T.reshape(-1, 2)
    tlc = tlc[tlc[:, 0] <= max_dim[0]]
    tlc = tlc[tlc[:, 1] <= max_dim[1]]
    
    windows = []
    for y,x in tlc.astype(int):
        windows.append(Window(x, y, patch_size[1], patch_size[0]))

    return windows

In [5]:
def add_ndvi(data, dtype_1=rio.float32):
    
    nir = data[3].astype(dtype_1)
    red = data[2].astype(dtype_1)

    # Allow division by zero
    np.seterr(divide='ignore', invalid='ignore')

    # Calculate NDVI
    ndvi = ((nir - red) / (nir + red)).astype(dtype_1)

    # Rescaling for use in 16bit output

    ndvi = (ndvi + 1) * (2**15 - 1)

    # Add NDVI band to end of array    
    rast = np.concatenate((data,[ndvi]),axis=0)
    
    rast = rast.astype(rio.uint16)
    
    return rast


# Download Model Files

In [3]:
model_url = "s3://canopy-production-ml/inference/model_files/model-best.h5"
weights_url = "s3://canopy-production-ml/inference/model_files/model_weights_best.h5"

download_model(model_url,weights_url)

In [6]:
model = load_model("model.h5","model_weights.h5") 

In [7]:
label_list = ["Industrial_agriculture","ISL","Mining","Roads","Shifting_cultivation"]

In [53]:
def output_windows(granule_dir,patch_size=100,
                   stride=100,SAVE=False,SAVE_INDIVIDUAL=False,
                   bands=[2, 3, 4, 8, 11, 12], 
                  model=model,
                   predict_thresh=.5,
                  label_list=label_list, 
                  job_name="test_inference_unwarped", 
                  output_filename="./inference_output/result.json"):
    
    granule_list = glob(f'{granule_dir}/*.tif')
    
    output_dict = {}
    
    granule_id_list = []
    
    window_id_list = []
    
    window_geom_list = []
    
    data_list = []
    
    label_master_list = []
    
    gdf_list = []
    
    timestamp = gen_timestamp()
    
    for j,granule_path in enumerate(granule_list[0:1]):
        
        granule_id = granule_path.split("/")[-1].split("_")[0]
    
        with rio.open(granule_path) as src:
            
            with WarpedVRT(src, crs='EPSG:3257', resampling=Resampling.nearest) as vrt:

                windows = get_windows(vrt.shape, (patch_size, patch_size), (stride, stride))

                for i, window in enumerate(windows):
            
                    print(f"predicting window {i + 1} of {len(windows)} of granulate {j + 1} of {len(granule_list)}",end='\r', flush=True)

                    label_name_list = []

                    window_id = i+1

                    data = vrt.read(bands,window=window, masked=True)

                    data = add_ndvi(data)

                    shape = data.shape

                    new_shape = (data.shape[0],patch_size,patch_size)

                    if shape != new_shape:

                        filled_array = np.full(new_shape, 0)
                        filled_array[:shape[0],:shape[1],:shape[2]] = data
                        data = filled_array
                        window = Window(window.col_off,window.row_off,shape[2],shape[1])


                    #image pre-processing / inference
                    prediction = model.predict(read_image_tf_out(data))
                    prediction = np.where(prediction > predict_thresh, 1, 0)
                    prediction_i = np.where(prediction == 1)[1]
                    for i in prediction_i:
                        label_name_list.append(label_list[i])

                    label_master_list.append(label_name_list)

                    #vectorizing raster bounds for visualization 
                    window_bounds = rio.windows.bounds(window, vrt.transform, height=patch_size, width=patch_size)
                    geom = box(*window_bounds)
                    geom_coords = list(geom.exterior.coords)
    #                 window_geom_list.append(geom)

                    #create or append to dict....

                    if granule_id in output_dict:

                        output_dict[granule_id].append({"window_id":window_id,"polygon_coords":geom_coords,"labels":label_name_list})

                    else:

                        output_dict[granule_id] = [{"window_id":window_id,"polygon_coords":geom_coords,"labels":label_name_list}]
            
            save_to_s3(output_dict,output_filename,job_name,timestamp)



    #             gdf = gpd.GeoDataFrame({"granule_id":granule_id_list,"window_id":window_id_list,"geometry":window_geom_list,"labels":label_master_list})
    #             gdf["labels"] = gdf["labels"].astype(str)

    #             gdf_list.append(gdf)
            
    return output_dict

In [54]:
# granule_dir = "./efs_inference_data/"
granule_dir = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Inference/granule_test(unwarped)"

output_dict = output_windows(granule_dir,output_filename="./inference_output/result-warped.json")

In [43]:
output_dict

{'1241': [{'window_id': 1,
   'polygon_coords': [(-11169700.787427548, -4193553.7119406024),
    (-11169700.787427548, -4191764.5839900365),
    (-11171489.915378114, -4191764.5839900365),
    (-11171489.915378114, -4193553.7119406024),
    (-11169700.787427548, -4193553.7119406024)],
   'labels': []},
  {'window_id': 2,
   'polygon_coords': [(-11167911.659476982, -4193553.7119406024),
    (-11167911.659476982, -4191764.5839900365),
    (-11169700.787427548, -4191764.5839900365),
    (-11169700.787427548, -4193553.7119406024),
    (-11167911.659476982, -4193553.7119406024)],
   'labels': []},
  {'window_id': 3,
   'polygon_coords': [(-11166122.531526417, -4193553.7119406024),
    (-11166122.531526417, -4191764.5839900365),
    (-11167911.659476982, -4191764.5839900365),
    (-11167911.659476982, -4193553.7119406024),
    (-11166122.531526417, -4193553.7119406024)],
   'labels': []},
  {'window_id': 4,
   'polygon_coords': [(-11164333.40357585, -4193553.7119406024),
    (-11164333.40357

In [166]:
data = output_dict

count = {}
label_match_results = []
granule_count = len(data.keys())
granule_list = data.keys()
count["granule_count"] = granule_count
for k1 in list(data.keys()):
    for i in range(len(data[k1])):
        if len(data[k1][i]['labels']) == 0:
            if "null_chips" not in count.keys():
                count["null_chips"] = 1
            else:
                count["null_chips"] += 1 
        for label in data[k1][i]['labels']:
            if label not in count.keys():
                count[label] = 1 
            else:
                    count[label] += 1 
        

In [167]:
count

{'granule_count': 1,
 'null_chips': 1512,
 'Shifting_cultivation': 336,
 'ISL': 82,
 'Roads': 5,
 'Industrial_agriculture': 1}

In [ ]:
for i in range (len(output_dict['101'])):
    print(output_dict['101'][i]['labels'])

In [ ]:
new_gdf.shape

In [ ]:
gdf.plot()

In [ ]:
new_gdf.to_file("./inference_output/test.geojson", driver='GeoJSON')

In [ ]:
gdf.to_file("./inference_output/test.geojson", driver='GeoJSON')

# Read Output Files

In [27]:
def process_output_files(json_path=None,download=False, filepath = "predict_test-2021-05-10-22-38-41.json", label_match="ISL"):

    s3 = boto3.resource('s3')

    #Download Model, Weights
    
    if download:
        
        bucket = json_path.split("/")[2]
        model_key = "/".join(json_path.split("/")[3:])
        filename = json_path.split("/")[-1]
        s3.Bucket(bucket).download_file(model_key, filename )
        filepath = filename
    
    with open(filepath) as jsonfile:
        data = json.load(jsonfile)
        

    count = {}
    label_match_results = []
    granule_count = len(data.keys())
    granule_list = data.keys()
    count["granule_count"] = granule_count
    for k1 in list(data.keys()):
        for i in range(len(data[k1])):
            if len(data[k1][i]['predicted_labels']) == 0:
                if "null_chips" not in count.keys():
                    count["null_chips"] = 1
                else:
                    count["null_chips"] += 1 
            for label in data[k1][i]['predicted_labels']:
                if label == label_match:
                    label_match_results.append([k1,data[k1][i]])
                if label not in count.keys():
                    count[label] = 1 
                else:
                    count[label] += 1 
    return count, label_match_results, granule_list, data

In [29]:
json_path = "s3://canopy-production-ml/inference/output/predict_3257-2021-05-15-22-18-23.json"

count, match_results, granule_list, data2 = process_output_files(download=False,
                                                                json_path=json_path,
                                                                filepath="/Users/purgatorid/Downloads/inference_output_test-2021-06-25-22-06-37.json")



In [37]:
data2["52"][0]

{'polygon_coords': [[-11746574.953781683, -4264220.997020786],
  [-11746574.953781683, -4262338.819566801],
  [-11748457.131235667, -4262338.819566801],
  [-11748457.131235667, -4264220.997020786],
  [-11746574.953781683, -4264220.997020786]],
 'predicted_labels': [],
 'actual_label': 'canopy-production-ml'}

In [40]:
match_results

[['52',
  {'polygon_coords': [[-11744692.776327698, -4296218.013738518],
    [-11744692.776327698, -4294335.836284533],
    [-11746574.953781681, -4294335.836284533],
    [-11746574.953781681, -4296218.013738518],
    [-11744692.776327698, -4296218.013738518]],
   'predicted_labels': ['ISL'],
   'actual_label': 'canopy-production-ml'}],
 ['52',
  {'polygon_coords': [[-11744692.776327698, -4256692.287204849],
    [-11744692.776327698, -4254810.109750864],
    [-11746574.953781681, -4254810.109750864],
    [-11746574.953781681, -4256692.287204849],
    [-11744692.776327698, -4256692.287204849]],
   'predicted_labels': ['ISL'],
   'actual_label': 'canopy-production-ml'}],
 ['52',
  {'polygon_coords': [[-11742810.598873714, -4275514.061744691],
    [-11742810.598873714, -4273631.884290706],
    [-11744692.776327698, -4273631.884290706],
    [-11744692.776327698, -4275514.061744691],
    [-11742810.598873714, -4275514.061744691]],
   'predicted_labels': ['ISL'],
   'actual_label': 'canopy-p

In [115]:
count

{'granule_count': 1, 'null_chips': 1}

In [55]:
sum(count.values()) - 97

128001

In [72]:
polygon_list = []
for result in match_results:
    coords = result[1]["polygon_coords"]
    polygon = Polygon(coords)
    polygon_list.append(polygon)

In [73]:
gdf = gpd.GeoDataFrame({"geometry":polygon_list})

In [74]:
gdf = gdf.set_crs(epsg=3257)
        
gdf = gdf.to_crs(epsg=4326)

In [75]:
gdf

,geometry
0,"POLYGON ((21.06129 -6.46611, 21.06975 -6.46911..."
1,"POLYGON ((21.00473 -6.59849, 21.01320 -6.60151..."
2,"POLYGON ((20.94185 -6.61425, 20.95032 -6.61727..."
3,"POLYGON ((20.90867 -6.44029, 20.91711 -6.44331..."
4,"POLYGON ((20.88146 -6.51632, 20.88991 -6.51935..."
5,"POLYGON ((20.83208 -6.44141, 20.84052 -6.44444..."


In [76]:
gdf.to_file("./inference_output/test_warped.geojson", driver='GeoJSON')

In [173]:
data['test.tif'][0]["polygon_coords"]

[[-11169700.787427548, -4193553.7119406024],
 [-11169700.787427548, -4191764.5839900365],
 [-11171489.915378114, -4191764.5839900365],
 [-11171489.915378114, -4193553.7119406024],
 [-11169700.787427548, -4193553.7119406024]]

# Get List of Non-Processed Granules

In [44]:
def s3_dir_ls(s3_dir_url):

        objs = []
        bucket = s3_dir_url.split("/")[2]
        key = "/".join(s3_dir_url.split("/")[3:5])

        s3 = boto3.resource('s3')
        my_bucket = s3.Bucket(bucket)


        for obj in my_bucket.objects.filter(Prefix=key):
            objs.append("s3://" + bucket + "/" + obj.key)

        return objs[1:]

s3_dir_url = "s3://canopy-production-ml/full_congo_basin/02.17.21_CB_GEE_Pull/"

all_granules = s3_dir_ls(s3_dir_url)


In [45]:
granule_ids_completed = list(data.keys())

In [46]:
def get_granule_paths(granule_ids_completed,all_granules):
    incomplete_granules = []
    for path in all_granules:
        granule_id = path.split("/")[-1].split("_")[0]
        if granule_id not in granule_ids_completed:
            incomplete_granules.append(path)
    return incomplete_granules
            
    

In [47]:
incomplete_gran_paths = get_granule_paths(granule_ids_completed,all_granules)

In [48]:
with open('incomplete_granules_3257.txt', 'w') as filehandle:
    for listitem in incomplete_gran_paths:
        filehandle.write('%s\n' % listitem)

# Output Vectorized Predicted Granules

In [ ]:
def s3_dir_match(s3_dir_url,granule_list):
    

    objs = []
    bucket = s3_dir_url.split("/")[2]
    key = "/".join(s3_dir_url.split("/")[3:5])

    s3 = boto3.resource('s3')
    my_bucket = s3.Bucket(bucket)

    window_geom_list = []
    granule_id_list = []
    for obj in my_bucket.objects.filter(Prefix=key):
        granule_id = obj.key.split("/")[-1].split("_")[0]
        if granule_id in granule_list:
            obj_url = "s3://" + bucket + "/" + obj.key
            with rio.open(obj_url) as src:
                bounds = src.bounds
                geom = box(*bounds)
                window_geom_list.append(geom)
                granule_id_list.append(granule_id)
    gdf = gpd.GeoDataFrame({"geometry":window_geom_list,"granule_id":granule_id_list})
                

    return gdf

In [ ]:
gdf = s3_dir_match("s3://canopy-production-ml/full_congo_basin/02.17.21_CB_GEE_Pull/",granule_list)

In [ ]:
gdf

In [ ]:
gdf.to_file("granules.json", driver="GeoJSON", index=True)

# Create and Export GDF of Original Labels Data

In [ ]:
FILE_NAME = "/Users/purgatorid/Downloads/polygons_021521.csv"


df = pd.read_csv(
    FILE_NAME)
gdf = gpd.GeoDataFrame(
    df,
    crs={'init': 'epsg:4326'})

In [ ]:
polygons = []
for polygon in df["polygon"]:
    polygons.append(Polygon(json.loads(polygon)["coordinates"][0]))

In [ ]:
gdf["geometry"] = polygons

In [ ]:
gdf.loc[90]

In [ ]:
gdf.to_file("output.json", driver="GeoJSON", index=True)

# Load and Reproject One Granulate Containing ISL 

In [ ]:
def convert_raster(input_file, dest_dir, epsg_format='EPSG:3257', windows=False):
    """Converts the rasters in the src_dir into a different EPSG format,
    keeping the same folder structure and saving them in the dest_dir."""
    
    print(input_file)

    filename = "test.tif"
#     print(filename)

    # If the respective grouping folders are not available 

    output_filepath = dest_dir + filename
    print(output_filepath)


#         Finally, we convert
    converted = gdal.Warp(output_filepath, [input_file],format='GTiff',
                          dstSRS=epsg_format, resampleAlg='near')
    converted = None
        
    print('Finished')

In [ ]:
granule = "/Users/purgatorid/Downloads/1241_full_congo_export_v12_all_bands_Feb_11_12_44_53_2021.tif"
dest_dir = "/Users/purgatorid/Downloads/"

convert_raster(granule,dest_dir)

# Visualize Results (Incomplete Code)

In [ ]:
def visualize_results(match_results,s3_url):
    for window in match_results:
        granule_id = window[0]
        

In [ ]:
t = {1,2,4}

# Running Without Windows Code - Direct Chip Predict 

In [108]:
model = load_model("model.h5","model_weights.h5") 

In [109]:
label_list = ["Industrial_agriculture","ISL","Mining","Roads","Shifting_cultivation"]

In [117]:
def output_predictions(granule_dir=None,patch_size=100,
                   stride=100,SAVE=False,SAVE_INDIVIDUAL=False,
                   bands=[2, 3, 4, 8, 11, 12], 
                  model=model,
                   predict_thresh=.5,
                  label_list=label_list, 
                  job_name="test_inference", 
                  output_filename="./inference_output/result.json", 
                      apply_windows=False, 
                      read_process="read_img_tf_out", 
                      sample_frac=1, 
                      granule_list=None):
    
    if granule_list is None:
        
        granule_list = glob(f'{granule_dir}/*.tif')
    
    end = len(granule_list) // sample_frac 
    
    granule_list = granule_list[0:end]
    
#     print(f"running inference on {len(granule_list)} chips")
    
    output_dict = {}
    
    granule_id_list = []
    
    window_id_list = []
    
    window_geom_list = []
    
    data_list = []
    
    label_master_list = []
    
    gdf_list = []
    
    timestamp = gen_timestamp()
    
    missed_chips = []
    
    for j,granule_path in enumerate(granule_list):
        
        label_name_list = []
        
        granule_id = granule_path.split("/")[-1].split("_")[0]
        filepath = granule_path.split("/")[-1]
        ground_label = granule_path.split("/")[2]
        
        print(f'Running inference on chip {j+1} of {len(granule_list)}',end='\r', flush=True)
        
        if filepath:

            with rio.open(granule_path) as src:

                data = src.read(bands,masked=True)

                data = add_ndvi(data)

                shape = data.shape

                if apply_windows:

                    new_shape = (data.shape[0],patch_size,patch_size)

                    if shape != new_shape:

                        filled_array = np.full(new_shape, 0)
                        filled_array[:shape[0],:shape[1],:shape[2]] = data
                        data = filled_array
                        window = Window(window.col_off,window.row_off,shape[2],shape[1])

                #image pre-processing / inference


                if read_process == "read_img_tf_out":
                    read_func = read_image_tf_out
                else:
                    read_func = read_image

                prediction = model.predict(read_func(data))
#                 print("original_prediction:",prediction)
                prediction = np.where(prediction > predict_thresh, 1, 0)
#                 print("sigmoid prediction gate:",prediction)
                prediction_i = np.where(prediction == 1)[1]
                if 1 not in np.where(prediction == 1)[1]:
                    missed_chips.append(granule_path)
#                 print("index of matching labels:",prediction_i)
                for i in prediction_i:
                    label_name_list.append(label_list[i])

#                 label_master_list.append(label_name_list)

                #vectorizing raster bounds for visualization 
                data_bounds = src.bounds
                geom = box(*data_bounds)
                geom_coords = list(geom.exterior.coords)
    #                 window_geom_list.append(geom)

                #create or append to dict....

                if granule_id in output_dict:

                    output_dict[granule_id].append({"polygon_coords":geom_coords,"predicted_labels":label_name_list,"actual_label":ground_label})

                else:

                    output_dict[granule_id] = [{"polygon_coords":geom_coords,"predicted_labels":label_name_list,"actual_label":ground_label}]
                    
                    
                save_to_s3(output_dict,output_filename,job_name,timestamp)
                    
#         if j % 1000 == 0: 

#             save_to_s3(output_dict,output_filename,job_name,timestamp)



    #             gdf = gpd.GeoDataFrame({"granule_id":granule_id_list,"window_id":window_id_list,"geometry":window_geom_list,"labels":label_master_list})
    #             gdf["labels"] = gdf["labels"].astype(str)

    #             gdf_list.append(gdf)

    return output_dict,missed_chips

In [161]:
# granule_dir = "./efs_inference_data/"
granule_dir_local = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Chips/misha_polygons_cloudfreemerge/yes/ISL/100/91/"
# granule_dir_efs = 

output_dict,missed_chips = output_predictions(granule_dir_local)

running inference on 75 chips


In [ ]:
df = pd.DataFrame({"file_path":missed_chips})

In [ ]:
df.to_csv("missed_chips.csv",index=False)

In [162]:
data = output_dict

count = {}
label_match_results = []
granule_count = len(data.keys())
granule_list = data.keys()
count["granule_count"] = granule_count
for k1 in list(data.keys()):
    for i in range(len(data[k1])):
        if len(data[k1][i]['labels']) == 0:
            if "null_chips" not in count.keys():
                count["null_chips"] = 1
            else:
                count["null_chips"] += 1 
        for label in data[k1][i]['labels']:
            if label not in count.keys():
                count[label] = 1 
            else:
                    count[label] += 1 
        

In [163]:
count

{'granule_count': 1,
 'ISL': 43,
 'Shifting_cultivation': 24,
 'Roads': 3,
 'null_chips': 3,
 'Industrial_agriculture': 2}

#### Opening Chips from S3

In [17]:
bucket = 'canopy-production-ml'

def s3_actions(bucket, in_path=None, out_path=None, copy_list=None, copy_dir=False, delete=False, stop_at=None, chip_list=False, chip_count=False):
    s3 = boto3.resource('s3')
    my_bucket = s3.Bucket(bucket)
    objs = []
    total_files = 0 
    

    
    if copy_dir or delete or chip_list or chip_count:
        for obj in my_bucket.objects.filter(Prefix=in_path):
            total_files += 1 
            objs.append(obj.key)
            if total_files == stop_at:
                break
        
    tot_objs = len(objs)
        
    if delete:
        for index,obj in enumerate(objs,1):
            print(f"deleting {index} of {tot_objs}", end='\r', flush=True)
            obj.delete()
            
    if chip_list:
        return objs
    
    if chip_count:
        return total_files
    
    if copy_list:
        total_copy_list = len(copy_list)

        for index,obj in enumerate(copy_list,1):
            
            print(f"copying {index} of {total_copy_list}", end='\r', flush=True)
            old_key = in_path + obj
            new_key = out_path + obj
            CopySource = {
                'Bucket': bucket,
                'Key':old_key}
            my_bucket.copy(CopySource, new_key)

In [18]:
path = 'chips/cloudfree-merge-polygons/dataset_v2/'

full_chip_list = s3_actions(bucket,in_path=path,chip_list=True)

In [42]:
prepend = "s3://canopy-production-ml/"

full_chip_list = [prepend + i for i in full_chip_list]

In [49]:
for x in full_chip_list:
    if ".csv" in x:
        full_chip_list.remove(x)
        continue

In [50]:
for x in full_chip_list:
    if ".csv" in x:
        print(x)

In [51]:
# s3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/

# rio.open(f"s3://canopy-production-ml/{full_chip_list[1]}")

#### Running Infernece on All Labeled Data

In [ ]:
# chips_dir_list = glob("./efs/*/100/*/*.tif")

In [ ]:
output_dict,missed_chips = output_predictions(granule_list=full_chip_list)

# Histogram for Numpy Array

In [ ]:
data1.shape

In [ ]:
def np_hist(arr,tensor=True):
    
    if tensor:
        arr = np.array(arr)
        arr = np.transpose(arr[0], (2, 1, 0))


    for i in range(arr.shape[0]):
        band_np = arr[i].flatten()
        plt.hist(band_np,label=str(i))


    plt.legend(prop={'size': 10})
    plt.show()
    

In [ ]:
np_hist(data)

In [ ]:
np_hist(data1,tensor=False)

# Retrieving Chips Missed on Inference Run

In [15]:
tot_len = 0 
for key in data.keys():
    tot_len += len(data[key])

In [16]:
tot_len

128001

In [56]:
len(full_chip_list)

129004

In [68]:
full_chip_list[1]

'chips/cloudfree-merge-polygons/dataset_v2/ISL/100/1/1_1000_1000.tif'

In [26]:
data["79"]

[{'polygon_coords': [[-11797353.520952346, -3216158.2367175575],
   [-11797353.520952346, -3214318.542869329],
   [-11799193.214800574, -3214318.542869329],
   [-11799193.214800574, -3216158.2367175575],
   [-11797353.520952346, -3216158.2367175575]],
  'predicted_labels': ['Shifting_cultivation'],
  'actual_label': 'ISL'},
 {'polygon_coords': [[-11812071.071738174, -3214318.542869329],
   [-11812071.071738174, -3212478.8490211004],
   [-11813910.765586402, -3212478.8490211004],
   [-11813910.765586402, -3214318.542869329],
   [-11812071.071738174, -3214318.542869329]],
  'predicted_labels': [],
  'actual_label': 'ISL'},
 {'polygon_coords': [[-11795513.827104116, -3214318.542869329],
   [-11795513.827104116, -3212478.8490211004],
   [-11797353.520952344, -3212478.8490211004],
   [-11797353.520952344, -3214318.542869329],
   [-11795513.827104116, -3214318.542869329]],
  'predicted_labels': [],
  'actual_label': 'ISL'},
 {'polygon_coords': [[-11813910.765586402, -3199600.9920834997],
   

In [104]:
def missing_chips(full_chip_list,data):
    
    completed_chips = []
    for key in list(data.keys()):
        for completed_chip in data[key]:
            completed_chip_id = (key,completed_chip["actual_label"])
            if completed_chip_id not in completed_chips:
                completed_chips.append(completed_chip_id)
                
    total_chips = []
    for chip in full_chip_list[1:]:
        try:
            gran_id = chip.split("/")[5]
            actual_label = chip.split("/")[3]
            total_chip_id = (gran_id,actual_label)
            if total_chip_id not in total_chips:
                total_chips.append(total_chip_id)
        except:
            continue
            
    missing_chips = list(set(total_chips) - set(completed_chips))
    chips_to_process = []
    
    
    for missing_chip_id in missing_chips:
        
        for chip in full_chip_list[1:]:
            try:
                gran_id = chip.split("/")[5]
                actual_label = chip.split("/")[3]
                total_chip_id = (gran_id,actual_label)
                if missing_chip_id == total_chip_id:
                    if total_chip_id not in chips_to_process:
                        chips_to_process.append(chip)
            except:
                continue
            
    return chips_to_process,missing_chips

In [105]:
chips_to_process,missing_chips = missing_chips(full_chip_list,data)

In [106]:
missing_chips

[('52', 'Shifting_cultivation'),
 ('92', 'Shifting_cultivation'),
 ('87', 'Shifting_cultivation')]

In [112]:
prepend = "s3://canopy-production-ml/"

missing_chips_list = [prepend + i for i in chips_to_process]

In [ ]:
output_dict,missed_chips = output_predictions(granule_list=missing_chips_list)

In [116]:
missing_chips_list

['s3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1000_1000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1000_1100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1000_1200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1000_1300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1000_1400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1000_1500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1000_1600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1000_1700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/datas

# Export True and False Positive on all 'ISL' labelled data

In [24]:
len(['s3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1000_1000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1000_1100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1000_1200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1000_1300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1000_1400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1000_1500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1000_1600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1000_1700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1000_1800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1000_1900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1000_2300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1000_2400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1000_2900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1000_3000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1000_3100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1000_500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1000_600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1000_700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1000_800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1000_900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1100_1000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1100_1100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1100_1200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1100_1300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1100_1400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1100_1500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1100_1600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1100_1700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1100_1800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1100_1900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1100_2300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1100_2400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1100_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1100_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1100_2700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1100_2900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1100_3000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1100_3100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1100_3200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1100_3300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1100_400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1100_500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1100_600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1100_700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1100_900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1200_1000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1200_1100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1200_1200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1200_1300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1200_1400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1200_1500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1200_1600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1200_1700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1200_1800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1200_1900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1200_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1200_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1200_2700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1200_2800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1200_2900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1200_300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1200_3000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1200_3100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1200_3200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1200_3300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1200_3400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1200_3500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1200_3600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1200_3700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1200_400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1200_500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1200_600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1200_700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1200_800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1200_900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1300_1000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1300_1100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1300_1200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1300_1300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1300_1400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1300_1500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1300_1800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1300_2800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1300_2900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1300_300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1300_3100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1300_3200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1300_3300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1300_3400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1300_3500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1300_3600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1300_3700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1300_3800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1300_3900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1300_400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1300_4000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1300_4100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1300_4200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1300_500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1300_600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1300_700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1300_800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1300_900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1400_1000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1400_1100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1400_1200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1400_2400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1400_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1400_3300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1400_3400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1400_3500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1400_3600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1400_3700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1400_3800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1400_3900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1400_400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1400_4000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1400_4100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1400_500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1400_600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1400_700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1400_800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1400_900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1500_1000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1500_1100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1500_1200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1500_2000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1500_2100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1500_2200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1500_2300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1500_2400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1500_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1500_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1500_2700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1500_3100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1500_3200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1500_3300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1500_3400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1500_3500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1500_3600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1500_3700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1500_3800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1500_3900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1500_4000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1500_4100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1500_500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1500_600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1500_700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1500_800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1600_1100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1600_1200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1600_1300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1600_1400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1600_2000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1600_2100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1600_2200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1600_2300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1600_2400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1600_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1600_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1600_2700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1600_2800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1600_3100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1600_3200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1600_3300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1600_3400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1600_3500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1600_3600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1600_3700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1600_3800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1600_3900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1600_4000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1600_600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1600_700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1700_1100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1700_1200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1700_1300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1700_1400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1700_1700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1700_1800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1700_1900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1700_2000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1700_2100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1700_2200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1700_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1700_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1700_2700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1700_3100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1700_3200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1700_3300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1700_3400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1700_3500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1700_3600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1700_3700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1700_3800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1700_3900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1800_1100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1800_1200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1800_1300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1800_1400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1800_1500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1800_1600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1800_1700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1800_1800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1800_1900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1800_2000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1800_2100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1800_2200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1800_2300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1800_2400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1800_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1800_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1800_2700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1800_2800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1800_3000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1800_3100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1800_3200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1800_3300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1800_3400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1800_3500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1800_3700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1800_3800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1800_3900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1900_1200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1900_1300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1900_1400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1900_1500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1900_1600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1900_1700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1900_1800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1900_1900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1900_2000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1900_2100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1900_2200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1900_2300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1900_2400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1900_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1900_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1900_2700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1900_2800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1900_2900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1900_3000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1900_3100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1900_3200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1900_3300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1900_3400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1900_3600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1900_3700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1900_3800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_1900_3900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_2000_1400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_2000_1500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_2000_1600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_2000_1700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_2000_1800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_2000_1900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_2000_2000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_2000_2100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_2000_2200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_2000_2300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_2000_2400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_2000_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_2000_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_2000_2700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_2000_2800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_2000_2900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_2000_3000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_2000_3100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_2000_3200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_2000_3300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_2000_3400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_2000_3600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_2000_3700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_2000_3800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_200_500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_200_600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_200_700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_2100_2000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_2100_2100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_2100_2200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_2100_2300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_2100_2400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_2100_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_2100_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_2100_2700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_2100_2800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_2100_2900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_2100_3000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_2100_3100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_2100_3200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_2100_3600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_2100_3700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_2200_2700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_2200_2800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_2200_3000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_300_1000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_300_400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_300_500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_300_600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_300_700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_300_800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_300_900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_400_1000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_400_1100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_400_400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_400_500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_400_600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_400_700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_400_800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_400_900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_500_1000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_500_1100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_500_1200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_500_1300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_500_400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_500_500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_500_600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_500_700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_500_800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_500_900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_600_1000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_600_1100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_600_1200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_600_1300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_600_1400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_600_400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_600_500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_600_600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_600_700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_600_800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_600_900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_700_1000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_700_1100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_700_1200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_700_1300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_700_1400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_700_1500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_700_1600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_700_1700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_700_400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_700_500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_700_600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_700_700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_700_800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_700_900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_800_1000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_800_1100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_800_1200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_800_1300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_800_1400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_800_1500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_800_1600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_800_1700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_800_1800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_800_1900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_800_400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_800_500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_800_600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_800_700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_800_800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_800_900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_900_1000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_900_1100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_900_1200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_900_1300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_900_1400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_900_1500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_900_1600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_900_1700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_900_1800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_900_1900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_900_400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_900_500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_900_600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_900_700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_900_800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/52/52_900_900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_1000_3300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_1000_3400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_1000_3500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_1000_3600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_1100_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_1100_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_1100_2700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_1100_3300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_1100_3400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_1100_3500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_1100_3600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_1200_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_1200_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_1200_2700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_1200_2800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_1300_2800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_1300_4800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_1400_4700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_1400_4800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_1400_4900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_1500_4700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_1500_4800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_1500_4900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_1600_4700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_1600_4800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_1600_4900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_1700_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_1700_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_1800_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_1800_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_1800_4500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_1800_4600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_1900_2400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_1900_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_1900_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_1900_2700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_1900_2800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_1900_4400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_1900_4500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_1900_4600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2000_2400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2000_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2000_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2000_2700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2000_2800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2000_4600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2000_4700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2100_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2100_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2100_2700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2100_2800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2200_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2200_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2200_2700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2300_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2300_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2300_2700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2300_2800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2400_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2400_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2400_2700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2400_2800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2500_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2500_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2500_2700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2500_2800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2600_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2600_2700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2600_2800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2700_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2700_2700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2700_4100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2700_4200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2700_4300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2700_4500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2800_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2800_2700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2800_4100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2800_4200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2800_4300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2800_4400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2800_4500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2900_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2900_2700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2900_4000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2900_4100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2900_4200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2900_4300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2900_4400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_2900_4500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3000_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3000_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3000_2700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3000_3900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3000_4000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3000_4100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3000_4200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3000_4300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3000_4400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3000_4500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3100_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3100_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3100_2700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3100_3800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3100_3900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3100_4000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3200_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3200_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3200_2700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3200_3800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3200_3900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3300_3800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3300_3900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3300_4000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3400_2400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3400_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3400_3800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3400_3900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3400_4000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3400_4100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3500_2400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3500_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3500_3800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3500_3900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3500_4000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3500_4100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3600_2400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3600_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3600_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3600_2700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3600_3800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3600_3900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3600_4000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3600_4100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3700_2400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3700_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3700_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3700_2700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3700_3700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3700_3800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3700_3900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3700_4000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3700_4100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3700_4200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3800_2400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3800_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3800_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3800_2700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3800_3900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3800_4000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3800_4100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3800_4200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3900_2400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3900_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3900_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3900_2700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3900_3900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3900_4000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3900_4100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3900_4200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3900_4400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_3900_4500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4000_2400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4000_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4000_2700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4000_2800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4000_4000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4000_4100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4000_4200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4000_4400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4000_4500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4100_2700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4100_2800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4100_4000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4100_4100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4100_4200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4100_4400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4200_2700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4200_2800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4200_2900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4300_2800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4300_2900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4400_2800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4400_2900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4400_3000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4500_2800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4500_2900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4500_3000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4500_3100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4600_2900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4600_3000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4600_3100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4600_3200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4700_2900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4700_3000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4700_3100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4700_3200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4700_4100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4700_4200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4800_3000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4800_3100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4800_4100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4800_4200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4900_2000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4900_2100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4900_2200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4900_2300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4900_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4900_2900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4900_3000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4900_3100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4900_4100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4900_4200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_4900_4300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5000_2200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5000_2300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5000_2400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5000_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5000_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5000_4100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5000_4200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5000_4300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5100_2400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5100_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5100_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5100_2700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5100_2800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5100_4100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5100_4200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5100_4300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5200_1900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5200_2000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5200_2100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5200_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5200_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5200_2700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5200_2800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5200_3700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5200_3800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5200_3900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5300_1900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5300_2000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5300_2100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5300_2200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5300_2300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5300_2400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5300_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5300_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5300_2700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5300_2800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5300_3700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5300_3800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5300_3900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5400_1600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5400_1700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5400_1800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5400_1900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5400_2000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5400_2100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5400_2200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5400_2300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5400_2400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5400_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5400_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5400_3700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5400_3800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5400_3900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5400_4000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5500_1600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5500_1700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5500_1800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5500_1900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5500_2000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5500_2100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5500_2200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5500_2300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5500_2400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5500_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5500_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5500_3800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5500_3900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5500_4000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5600_1600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5600_1700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5600_1800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5600_1900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5600_2000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5600_2100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5600_2200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5600_2300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5600_2400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5600_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5600_3400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5600_3500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5600_3700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5600_3800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5600_3900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5600_4000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5600_4100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5700_1700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5700_1800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5700_1900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5700_2000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5700_2100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5700_2200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5700_2300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5700_2400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5700_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5700_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5700_3300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5700_3400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5700_3500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5700_3600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5700_3700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5700_3800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5700_3900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5700_4000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5800_1800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5800_1900.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5800_2000.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5800_2100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5800_2200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5800_2300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5800_2400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5800_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5800_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5800_3300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5800_3500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5800_3600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5800_3700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5800_3800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5900_2300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5900_2400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5900_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5900_3300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5900_3400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5900_3500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_5900_3700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6000_2300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6000_2400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6000_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6000_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6000_3300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6000_3400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6000_3500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6000_3600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6000_3700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6000_3800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_600_2400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_600_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_600_3500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_600_3600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_600_3700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6100_2200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6100_2300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6100_2400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6100_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6100_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6100_2700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6100_3300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6100_3400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6100_3500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6100_3600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6100_3700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6200_2100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6200_2200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6200_2300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6200_2400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6200_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6200_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6200_2700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6200_3500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6200_3600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6300_2200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6300_2300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6300_2400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6300_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6300_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6300_3300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6300_3400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6300_3500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6300_3600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6400_2300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6400_2400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6400_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6400_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6400_3300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6400_3400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6400_3500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6400_3600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6400_3700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6400_3800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6500_2400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6500_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6500_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6500_3300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6500_3400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6500_3500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6500_3600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6500_3700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6500_3800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6600_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6600_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6600_3300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6600_3400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6600_3500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6600_3600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6600_3700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6600_3800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6700_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6700_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6700_2700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6700_3300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6700_3400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6700_3500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6700_3600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6700_3700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6700_3800.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6800_2600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6800_3300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6800_3400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6800_3500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_6800_3600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_7000_3500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_7000_3600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_7000_3700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_700_2400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_700_2500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_700_3400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_700_3500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_700_3600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_700_3700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_7100_3500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_7100_3600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_7100_3700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_7200_3400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_7200_3500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_7200_3600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_7200_3700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_7300_3700.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_7500_3500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_7500_3600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_7600_3500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_7600_3600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_7700_3600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_800_3500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_800_3600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_900_3300.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_900_3400.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_900_3500.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/92/92_900_3600.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/87/87_0_100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/87/87_0_200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/87/87_100_0.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/87/87_100_100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/87/87_100_200.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/87/87_200_100.tif',
 's3://canopy-production-ml/chips/cloudfree-merge-polygons/dataset_v2/Shifting_cultivation/100/87/87_200_200.tif'])

835

In [20]:
len(data["87"])

41

In [21]:
len(data2["87"])

7

52 1034
52 377
92 4543
92 451
87 41
87 7

In [32]:
for key in data2.keys():
    for polygon in data2[key]:
        data[key].append(polygon)

In [13]:
data.keys()

dict_keys(['79', '88', '26', '62', '44', '35', '53', '17', '22', '40', '13', '48', '84', '39', '75', '93', '99', '20', '2', '11', '64', '46', '82', '37', '73', '55', '91', '24', '42', '33', '15', '68', '77', '59', '1', '100', '89', '36', '72', '54', '90', '41', '32', '5', '85', '76', '94', '21', '30', '3', '29', '65', '47', '83', '38', '74', '25', '43', '34', '70', '16', '80', '71', '8', '31', '4', '66', '57', '101', '28', '19', '60', '6', '51', '86', '10', '27', '63', '45', '81', '9', '18', '23', '50', '14', '67', '49', '58', '12', '56', '92', '61', '7', '52', '69', '87', '78'])

In [44]:
for key in data2.keys():
    print(key,len(data[key]))
    print(key,len(data2[key]))
#     for polygon in data2[key]:
#         data[key].append(polygon)

52 1411
52 377
92 4994
92 451
87 48
87 7


In [75]:
data.keys()

dict_keys(['79', '88', '26', '62', '44', '35', '53', '17', '22', '40', '13', '48', '84', '39', '75', '93', '99', '20', '2', '11', '64', '46', '82', '37', '73', '55', '91', '24', '42', '33', '15', '68', '77', '59', '1', '100', '89', '36', '72', '54', '90', '41', '32', '5', '85', '76', '94', '21', '30', '3', '29', '65', '47', '83', '38', '74', '25', '43', '34', '70', '16', '80', '71', '8', '31', '4', '66', '57', '101', '28', '19', '60', '6', '51', '86', '10', '27', '63', '45', '81', '9', '18', '23', '50', '14', '67', '49', '58', '12', '56', '92', '61', '7', '52', '69', '87', '78'])

In [49]:
def output_for_gdf(data,label_match="ISL"):
    count = {}
    label_match_results = []
    granule_count = len(data.keys())
    granule_list = data.keys()
    count["granule_count"] = granule_count
    for k1 in list(data.keys()):
        for i in range(len(data[k1])):
            if len(data[k1][i]['predicted_labels']) == 0:
                if "null_chips" not in count.keys():
                    count["null_chips"] = 1
                else:
                    count["null_chips"] += 1 
            for label in data[k1][i]['predicted_labels']:
                if label == label_match:
                    label_match_results.append([k1,data[k1][i]])
                
    return label_match_results

In [50]:
data_for_gdf = output_for_gdf(data)

In [69]:
data_for_gdf[0][1]["actual_label"]

'ISL'

In [71]:
predicted_labels = []
actual_labels = []
polygon_list = []

for result in data_for_gdf:
    predicted_labels.append(result[1]["predicted_labels"][0])
    actual_labels.append(result[1]["actual_label"])
    coords = result[1]["polygon_coords"]
    polygon = Polygon(coords)
    polygon_list.append(polygon)

In [111]:
gdf = gpd.GeoDataFrame({"predicted_label":predicted_labels,"actual_label":actual_labels,"geometry":polygon_list})

In [112]:
gdf.shape

(3717, 3)

In [113]:
false_positive_gdf = gpd.GeoDataFrame(gdf[gdf["predicted_label"] != gdf["actual_label"] ]["geometry"]).reset_index(drop=True)

In [114]:
true_positive_gdf = gpd.GeoDataFrame(gdf[gdf["predicted_label"] == gdf["actual_label"] ]["geometry"]).reset_index(drop=True)

In [115]:
false_positive_gdf.shape

(663, 1)

In [116]:
true_positive_gdf.shape

(3054, 1)

In [117]:
def convert_to_4326(gdf):

    gdf = gdf.set_crs(epsg=3257)
    gdf = gdf.to_crs(epsg=4326)
    return gdf

In [118]:
false_positive_gdf = convert_to_4326(false_positive_gdf)
true_positive_gdf = convert_to_4326(true_positive_gdf)

In [119]:
false_positive_gdf

,geometry
0,"POLYGON ((14.31550 3.26610, 14.32356 3.26215, ..."
1,"POLYGON ((14.37151 3.27875, 14.37958 3.27481, ..."
2,"POLYGON ((14.37565 3.26670, 14.38371 3.26276, ..."
3,"POLYGON ((14.34361 3.26234, 14.35167 3.25839, ..."
4,"POLYGON ((14.29938 3.27401, 14.30744 3.27005, ..."
...,...
658,"POLYGON ((15.83476 0.97153, 15.84294 0.96777, ..."
659,"POLYGON ((16.03105 1.10016, 16.03923 1.09644, ..."
660,"POLYGON ((15.97912 0.98490, 15.98731 0.98116, ..."
661,"POLYGON ((16.27388 0.96029, 16.28209 0.95658, ..."


In [120]:
true_positive_gdf

,geometry
0,"POLYGON ((25.61819 1.07992, 25.62684 1.07756, ..."
1,"POLYGON ((25.56952 1.03713, 25.57817 1.03476, ..."
2,"POLYGON ((25.60955 1.08227, 25.61819 1.07992, ..."
3,"POLYGON ((25.59695 1.10439, 25.60559 1.10203, ..."
4,"POLYGON ((25.66213 1.10531, 25.67077 1.10296, ..."
...,...
3049,"POLYGON ((16.00932 3.97596, 16.01749 3.97224, ..."
3050,"POLYGON ((15.96256 3.93791, 15.97074 3.93418, ..."
3051,"POLYGON ((15.93361 3.96103, 15.94177 3.95730, ..."
3052,"POLYGON ((15.98111 3.97892, 15.98928 3.97520, ..."


In [122]:
true_positive_gdf.to_file("true_positive_ISL_labelled.geojson", driver="GeoJSON", index=True)

In [123]:
false_positive_gdf.to_file("false_positive_ISL_labelled.geojson", driver="GeoJSON", index=True)